In [107]:
# The following code is only for Google Colab.
# If you are running this notebook locally, you should not run this cell.

# %pip install google-colab

# from google.colab import drive
# drive.mount('/content/drive/')

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import zipfile
from pathlib import Path
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')



[nltk_data] Downloading package punkt to /Users/vongoc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vongoc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vongoc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/vongoc/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vongoc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/vongoc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [109]:
current_directory = Path(__file__).parent if '__file__' in locals() else Path.cwd()
print(f"Current directory: {current_directory}")
with zipfile.ZipFile(f"{current_directory}/data.zip", 'r') as zip_ref:
    zip_ref.extractall(current_directory)

Current directory: /Users/vongoc/Documents/GitHub/ASM3-APDS


In [110]:
df = pd.read_csv('data/assignment3.csv')
df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
...,...,...,...,...,...,...,...,...,...,...
19657,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
19658,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
19659,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
19660,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [111]:
# 2/3/4. Tokenizing clothing review

# define tokenizer
tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")

# Tokenize the review text + lowercasing and removing words with length < 2
df["tokens"] = (
    df["Review Text"]
      .fillna("") # replace NaN with empty string
      .astype(str)
      .apply(lambda text: [token.lower() for token in tokenizer.tokenize(text) if len(token) > 1])
)

# quick spot-checks (delete when done)
print(df["tokens"].head(3).tolist())
assert isinstance(df["tokens"].iloc[0], list)

[['had', 'such', 'high', 'hopes', 'for', 'this', 'dress', 'and', 'really', 'wanted', 'it', 'to', 'work', 'for', 'me', 'initially', 'ordered', 'the', 'petite', 'small', 'my', 'usual', 'size', 'but', 'found', 'this', 'to', 'be', 'outrageously', 'small', 'so', 'small', 'in', 'fact', 'that', 'could', 'not', 'zip', 'it', 'up', 'reordered', 'it', 'in', 'petite', 'medium', 'which', 'was', 'just', 'ok', 'overall', 'the', 'top', 'half', 'was', 'comfortable', 'and', 'fit', 'nicely', 'but', 'the', 'bottom', 'half', 'had', 'very', 'tight', 'under', 'layer', 'and', 'several', 'somewhat', 'cheap', 'net', 'over', 'layers', 'imo', 'major', 'design', 'flaw', 'was', 'the', 'net', 'over', 'layer', 'sewn', 'directly', 'into', 'the', 'zipper', 'it'], ['love', 'love', 'love', 'this', 'jumpsuit', "it's", 'fun', 'flirty', 'and', 'fabulous', 'every', 'time', 'wear', 'it', 'get', 'nothing', 'but', 'great', 'compliments'], ['this', 'shirt', 'is', 'very', 'flattering', 'to', 'all', 'due', 'to', 'the', 'adjustable

In [112]:
# 5. Remove stopwords using the provided stop words list (i.e., stopwords_en.txt). It is located inside the same downloaded folder. 
# Find file stopwords in data
stop_candidates = [Path("stopwords_en.txt"), Path("data/stopwords_en.txt")]
stop_path = next((p for p in stop_candidates if p.exists()), None)
assert stop_path is not None, "stopwords_en.txt not found. Please put it in project root or inside data/."

# Load stopwords
with open(stop_path, "r", encoding="utf-8") as f:
    STOPWORDS = {line.strip() for line in f if line.strip()}

print(f"[Q5] Loaded {len(STOPWORDS)} stopwords from {stop_path}")

# Remove stopwords (Compare lowercase)
def remove_stopwords(tokens, stopset):
    return [t for t in tokens if t.lower() not in stopset]

df["tokens"] = df["tokens"].apply(lambda xs: remove_stopwords(xs, STOPWORDS))

# Print to check
print(df[["tokens"]].head(2))
print("df['tokens'] ready for Step 6 & 7.")

[Q5] Loaded 570 stopwords from data/stopwords_en.txt
                                              tokens
0  [high, hopes, dress, wanted, work, initially, ...
1  [love, love, love, jumpsuit, fun, flirty, fabu...
df['tokens'] ready for Step 6 & 7.


In [113]:
# 6. Remove the word that appears only once in the document collection, based on term frequency.
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
 
df["tokens"] = df["tokens"].apply(lambda tokens: [token for token in tokens if term_freq[token] > 1])

# Double check
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
term_freq

dress          9334
size           7860
love           7722
fit            6582
top            6542
               ... 
upsize            2
relaxed-fit       2
undressed         2
connected         2
crosswrap         2
Name: count, Length: 7549, dtype: int64

In [114]:

# 7. Remove the top 20 most frequent words based on document frequency. 
top_20_words = term_freq.nlargest(20).index
df["tokens"] = df["tokens"].apply(lambda tokens: [token for token in tokens if token not in top_20_words])

# Double check
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
term_freq

fits           2541
beautiful      2491
large          2485
material       2438
length         2366
               ... 
upsize            2
relaxed-fit       2
undressed         2
connected         2
crosswrap         2
Name: count, Length: 7529, dtype: int64

In [115]:
stemmer = PorterStemmer()
df["stemmed_tokens"] = df["tokens"].apply(lambda tokens: [stemmer.stem(t) for t in tokens])

# -------------------------------
# POS-aware Lemmatization
# -------------------------------
lemmatizer = WordNetLemmatizer()

# Map POS tags to WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

def lemmatize_tokens(tokens):
    pos_tags = nltk.pos_tag(tokens)
    return [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]

df["lemmatized_tokens"] = df["tokens"].apply(lemmatize_tokens)




In [116]:
print(df[["tokens", "stemmed_tokens", "lemmatized_tokens"]].head(20))

                                               tokens  \
0   [high, hopes, wanted, work, initially, petite,...   
1   [jumpsuit, fun, flirty, fabulous, time, compli...   
2   [shirt, due, adjustable, front, tie, length, l...   
3   [tracy, reese, dresses, petite, feet, tall, br...   
4   [basket, hte, person, store, pick, teh, pale, ...   
5   [carbon, store, pick, ton, stuff, pair, skirts...   
6   [xs, runs, snug, bust, feminine, usual, retail...   
7   [lbs, petite, make, length, long, typically, x...   
8   [runs, esp, zipper, area, runs, sp, typically,...   
9   [find, reviews, written, savvy, shoppers, past...   
10  [black, xs, larkspur, midi, bother, lining, sk...   
11  [choice, holiday, gatherings, length, knee, co...   
12  [package, wanted, badly, put, hour-glass, figu...   
13  [material, leg, opening, large, length, hits, ...   
14  [chance, blouse, glad, crazy, blouse, photogra...   
15  [super, cozy, coat, work, cold, dry, days, goo...   
16  [feel, tulle, year's, eve, 

In [117]:
df.to_csv(f"processed.csv", index=False)

In [118]:
# 9. Build a vocabulary of the cleaned/processed reviews, and save it in a txt file 
all_tokens = [token for tokens in df["lemmatized_tokens"] for token in tokens]
vocabulary = sorted(set(all_tokens))
vocab_dict = {word: i for i, word in enumerate(vocabulary)}

# Save to file
with open("vocabulary.txt", "w", encoding="utf-8") as f:
    for word, index in vocab_dict.items():
        f.write(f"{word}:{index}\n")

print("Preprocessing done. Vocabulary saved to vocabulary.txt")

Preprocessing done. Vocabulary saved to vocabulary.txt
